In [14]:
from datasets import load_dataset

import vertexai
from vertexai.generative_models import GenerativeModel
import vertexai.preview.generative_models as generative_models

seed =42

In [12]:
QPM = 3
PROJECT = "amir-genai-bb"
LOCATION = "us-central1"

In [39]:

def gemini_label(prompt):
    vertexai.init(project="amir-genai-bb", location="us-central1")
    model = GenerativeModel("gemini-1.0-pro-002",
                           system_instruction=[
                             "You are a helpful data labeler.",
                             "Your mission is to label the input data based on the instruction you will receive.",
                             ],
                        )
    responses = model.generate_content(
      [prompt],
      generation_config=generation_config,
      safety_settings=safety_settings,
      stream=False,
  )
    return(responses.text)


generation_config = {
    "max_output_tokens": 2048,
    "temperature": 1,
    "top_p": 1,
}

safety_settings = {
    generative_models.HarmCategory.HARM_CATEGORY_HATE_SPEECH: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    generative_models.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    generative_models.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    generative_models.HarmCategory.HARM_CATEGORY_HARASSMENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
}


In [64]:
gemini_prompt_template = """
<data_description>
{description}
</data_description>
-----------

<context>
{datapoint}
</context>
------------

<labels>
{labels}
</labels>
------------

INSTRUCTION:
- familirize yourself with the data using data_description
- read the context carefully. this is the data point you need to label.
- take your time and label the dadatapoint with the most appropriate option using the provided labels.
- return the result as a single label from the <labels>. Don't provide explanations
"""

In [68]:
dataset = load_dataset("yelp_polarity", split="train") # https://huggingface.co/datasets/yelp_polarity

# take a small sample for dev purposes
dataset_sample = dataset.shuffle(seed=seed).select(range(1000))

# user provided data description
DESCRIPTION = """
This is a dataset for binary sentiment classification.
It contains highly polar yelp reviews.
Negative polarity is class 0, and positive class 1.
"""
i = 100
datapoint = dataset_sample["text"][i]

LABEL_SET = [0, 1] 

In [69]:
prompt = gemini_prompt_template.format(description= DESCRIPTION,
                              datapoint=datapoint,
                              labels=LABEL_SET)


gemini_label(prompt), dataset_sample["label"][i]

('0', 0)